# Conexión a Bases de datos

**Autor:** Roberto Muñoz <br />
**E-mail:** <rmunoz@metricarts.com> <br />
**Github:** <https://github.com/rpmunoz> <br />


- Instalar el conector de MySQL para python

    `!pip install mysql-connector-python`
    
- Crear usuario en DB y dar permisos de lectura

 `GRANT SELECT ON chile.* TO 'user'@'%' IDENTIFIED BY 'metric2019';`
 
 `GRANT SELECT ON censo_2017.* TO 'user'@'%' IDENTIFIED BY 'metric2019';`
 
 `GRANT SELECT ON chile.* TO 'user'@'localhost' IDENTIFIED BY 'metric2019';`

In [ ]:
#!pip install mysql-connector-python

In [ ]:
import numpy as np
import pandas as pd
import mysql.connector as sql

# Análisis de división geográfica en Chile

In [ ]:
# Database args
db_user = 'user@tallerpython'
db_passwd = 'metric2019'
db_host = 'tallerpython.mysql.database.azure.com'
db_name = 'chile'

In [ ]:
cnx = sql.connect(user=db_user, password=db_passwd, host=db_host, database=db_name)

Usando el cursor de SQL. Ojo que antes de cerrar el cursor tenemos que realizar las operaciones de lectura que necesitemos.

In [ ]:
query= "SHOW TABLES;"

cursor = cnx.cursor(buffered=True)
cursor.execute(query)

for el in cursor:
    print(el)

cursor.close()

Usando el método read_sql de Pandas

In [ ]:
query= "SHOW TABLES;"
tablas=pd.read_sql(query, con=cnx)
tablas

Hagamos un select en la tabla regiones

In [ ]:
query= "SELECT * from regiones;"
regiones=pd.read_sql(query, con=cnx)
regiones

La siguiente consulta está parametrizada. Fíjese cómo se declaran los parámetros dentro de la cadena de la consulta, y cómo después se ingresan, en el comando read_sql de Pandas.

In [ ]:
query= "SELECT * from regiones WHERE region_id BETWEEN %s and %s;"
regiones=pd.read_sql(query, params=(2,8), con=cnx)
regiones.head()

Como verá en la siguiente consulta, estamos utilizando cosas aprendidas en los tutoriales anteriores. Estamos armando dinámicamente la cadena que respresenta una consulta. Específicamente, utilizamos una lista con id's de regiones que nos interesan, lo convertimos a un objeto `array` de numpy (np.array), después aplicamos la función `map` para aplicar la función `str` al array, y finalmente, concatenamos los resultados con el caracter ",". **Todo en una línea**.
¿En cuantas líneas podría hacer esto en Java?

In [ ]:
','.join(["Hola","Mundo","Como","Estan"])

In [ ]:
region_ids=[5,8,10]
query_region= ', '.join(map(str, np.array(region_ids)))
query_region

In [ ]:
region_ids=[5,8,10]

query= "SELECT * from regiones WHERE region_id IN ("
query_region= ', '.join(map(str, np.array(region_ids)))
query= query + query_region + ");"
print(query)

regiones=pd.read_sql(query, con=cnx)
regiones

## Uso de pandas.merge()

In [ ]:
query= "SELECT * from regiones;"
regiones=pd.read_sql(query, con=cnx)
regiones

In [ ]:
query= "SELECT * from provincias;"
provincias=pd.read_sql(query, con=cnx)
provincias.head()

In [ ]:
query= "SELECT * from comunas;"
comunas=pd.read_sql(query, con=cnx)
comunas.head()

In [ ]:
regiones_provincias=pd.merge(regiones, provincias, how='outer')
regiones_provincias.head()

Cerramos la conexión al servidor de MySQL.

In [ ]:
cnx.close()

# Análisis los datos del Censo 2017

In [ ]:
# Database args
db_user = 'user@tallerpython'
db_passwd = 'metric2019'
db_host = 'tallerpython.mysql.database.azure.com'
db_name = 'censo_2017'

In [ ]:
cnx = sql.connect(user=db_user, password=db_passwd, host=db_host, database=db_name)

In [ ]:
query= "SHOW TABLES;"
tablas=pd.read_sql(query, con=cnx)
tablas

In [ ]:
query= "DESCRIBE comunas;"
censo_describe=pd.read_sql(query, con=cnx)
censo_describe

In [ ]:
query= "SELECT count(*) from comunas;"
censo_count=pd.read_sql(query, con=cnx)
censo_count.head()

In [ ]:
query= "SELECT * from comunas;"
censo_df = pd.read_sql(query, con=cnx)
censo_df.head()

### Sumamos la poblacion en Chile por sexo

In [ ]:
censo_df['hombres'].sum()

In [ ]:
censo_df['mujeres'].sum()

### Extraemos la poblacion de la region metropolitana

In [ ]:
censo_df['nombre region'].unique()

In [ ]:
censo_rm_df = censo_df[censo_df['nombre region'] == 'metropolitana de santiago']
censo_rm_df.head()

Cerramos la conexión al servidor de MySQL. **¡¡No olvide hacer esto!!**

In [ ]:
cnx.close()

## Ejercicios

### 1. ¿Cuáles son las 5 comunas de la Región metropolitana con mayor nñumero de habitantes?

### 2. ¿Cuál es la región de Chile con el mayor porcentaje de población rural?

